# Visualization project

In [2]:
#imports
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
from dash import Dash, html, dcc
os.chdir('C:\\Users\\filip\\Desktop\\visualization')
os.getcwd()


'C:\\Users\\filip\\Desktop\\visualization'

In [3]:
high_popularity = pd.read_csv("high_popularity_spotify_data.csv")
#high_popularity.head()

In [4]:
low_popularity = pd.read_csv("low_popularity_spotify_data.csv")
#low_popularity.head()

In [5]:
#mege
high_popularity['popularity'] = 'High'
low_popularity['popularity'] = 'Low'
df = pd.concat([high_popularity, low_popularity], ignore_index=True)
#pre processing
df['year'] = df['track_album_release_date'].str[:4].astype('int')
df['track_album_release_date'] = pd.to_datetime(df['track_album_release_date'], errors='coerce')
df['decade'] = ((df['year'] // 10) * 10).astype('str').str[2:]+'s'# good for buttons

# Function to turn "90s", "00s" into real years for sorting
def sort_key(d):
    # Removes the 's' and convert to number (e.g., "90" -> 90)
    val = int(d[:-1])
    # If it's small  it belongs to 2000s
    if val < 30: 
        return val + 2000
    # Otherwise it belongs to 1900s
    else: 
        return val + 1900

# Now get the unique decades and sort them using our key
decades_list = sorted(df['decade'].unique(), key=sort_key)

print("Correct Order:", decades_list)


Correct Order: ['50s', '60s', '70s', '80s', '90s', '00s', '10s', '20s']


In [6]:
df.head(100)

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,track_popularity,track_href,uri,track_album_name,playlist_name,analysis_url,track_id,track_name,track_album_release_date,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id,popularity,year,decade
0,0.592,157.969,0.521,pop,-7.777,0.1220,0.535,"Lady Gaga, Bruno Mars",3.0,0.0304,100,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,spotify:track:2plbrEY59IikOBgBGLjaoe,Die With A Smile,Today's Top Hits,https://api.spotify.com/v1/audio-analysis/2plb...,2plbrEY59IikOBgBGLjaoe,Die With A Smile,2024-08-16,0.000000,10FLjwfpbxLmW8c25Xyc2N,0.0,6.0,251668.0,0.308000,2plbrEY59IikOBgBGLjaoe,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M,High,2024,20s
1,0.507,104.978,0.747,pop,-10.171,0.1170,0.438,Billie Eilish,4.0,0.0358,97,https://api.spotify.com/v1/tracks/6dOtVTDdiauQ...,spotify:track:6dOtVTDdiauQNBQEDOtlAB,HIT ME HARD AND SOFT,Today's Top Hits,https://api.spotify.com/v1/audio-analysis/6dOt...,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,2024-05-17,0.060800,7aJuG4TFXa2hmE4z1yxc3n,1.0,2.0,210373.0,0.200000,6dOtVTDdiauQNBQEDOtlAB,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M,High,2024,20s
2,0.808,108.548,0.554,pop,-4.169,0.1590,0.372,Gracie Abrams,4.0,0.0368,93,https://api.spotify.com/v1/tracks/7ne4VBA60CxG...,spotify:track:7ne4VBA60CxGM75vw0EYad,The Secret of Us (Deluxe),Today's Top Hits,https://api.spotify.com/v1/audio-analysis/7ne4...,7ne4VBA60CxGM75vw0EYad,That’s So True,2024-10-18,0.000000,0hBRqPYPXhr1RkTDG3n4Mk,1.0,1.0,166300.0,0.214000,7ne4VBA60CxGM75vw0EYad,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M,High,2024,20s
3,0.910,112.966,0.670,pop,-4.070,0.3040,0.786,Sabrina Carpenter,4.0,0.0634,81,https://api.spotify.com/v1/tracks/1d7Ptw3qYcfp...,spotify:track:1d7Ptw3qYcfpdLNL5REhtJ,Short n' Sweet,Today's Top Hits,https://api.spotify.com/v1/audio-analysis/1d7P...,1d7Ptw3qYcfpdLNL5REhtJ,Taste,2024-08-23,0.000000,4B4Elma4nNDUyl6D5PvQkj,0.0,0.0,157280.0,0.093900,1d7Ptw3qYcfpdLNL5REhtJ,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M,High,2024,20s
4,0.783,149.027,0.777,pop,-4.477,0.3550,0.939,"ROSÉ, Bruno Mars",4.0,0.2600,98,https://api.spotify.com/v1/tracks/5vNRhkKd0yEA...,spotify:track:5vNRhkKd0yEAg8suGBpjeY,APT.,Today's Top Hits,https://api.spotify.com/v1/audio-analysis/5vNR...,5vNRhkKd0yEAg8suGBpjeY,APT.,2024-10-18,0.000000,2IYQwwgxgOIn7t3iF6ufFD,0.0,0.0,169917.0,0.028300,5vNRhkKd0yEAg8suGBpjeY,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M,High,2024,20s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.808,151.008,0.577,rock,-7.040,0.2210,0.869,The Police,4.0,0.0390,75,https://api.spotify.com/v1/tracks/1oYYd2gnWZYr...,spotify:track:1oYYd2gnWZYrt89EBXdFiO,Reggatta De Blanc (Remastered 2003),Rock Classics,https://api.spotify.com/v1/audio-analysis/1oYY...,1oYYd2gnWZYrt89EBXdFiO,Message In A Bottle,1979-10-02,0.000013,2EpuND32cO7CX0gXZl2NB6,0.0,1.0,290280.0,0.033800,1oYYd2gnWZYrt89EBXdFiO,classic,audio_features,37i9dQZF1DWXRqgorJj26U,High,1979,70s
96,0.758,112.113,0.489,rock,-6.491,0.0920,0.435,David Bowie,4.0,0.0297,72,https://api.spotify.com/v1/tracks/7Jh1bpe76CNT...,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,"""Heroes"" (2017 Remaster)",Rock Classics,https://api.spotify.com/v1/audio-analysis/7Jh1...,7Jh1bpe76CNTCgdgAdBw4Z,Heroes - 2017 Remaster,1977-01-01,0.490000,4I5zzKYd2SKDgZ9DRf5LVk,1.0,7.0,371413.0,0.000678,7Jh1bpe76CNTCgdgAdBw4Z,classic,audio_features,37i9dQZF1DWXRqgorJj26U,High,1977,70s
97,0.932,112.937,0.564,rock,-10.068,0.3490,0.619,Led Zeppelin,4.0,0.0554,75,https://api.spotify.com/v1/tracks/78lgmZwycJ3n...,spotify:track:78lgmZwycJ3nzsdgmPPGNx,Led Zeppelin III (Remaster),Rock Classics,https://api.spotify.com/v1/audio-analysis/78lg...,78lgmZwycJ3nzsdgmPPGNx,Immigrant Song - Remaster,1970-01-01,0.169000,6P5QHz4XtxOmS5EuiGIPut,1.0,11.0,146250.0,0.013000,78lgmZwycJ3nzsdgmPPGNx,classic,aud

## Processing for gengre decade changes

In [8]:
# 1. counts songs per genre per decade
genre_counts = df.groupby(['decade', 'playlist_genre']).size().reset_index(name='count')

# 2. Sort so they are together
genre_counts = genre_counts.sort_values(['playlist_genre', 'decade'])
print(genre_counts)
# 3. Calculates the delta
# We group by genre and take the difference from the previous row
genre_counts['prev_count'] = genre_counts.groupby('playlist_genre')['count'].shift(1)
genre_counts['change'] = genre_counts['count'] - genre_counts['prev_count'].fillna(0)

# Now 'genre_counts' has a 'change' column! 
# +50 means it grew by 50 songs. -20 means it shrank.## Processing for genre decade changes

    decade playlist_genre  count
48     20s      afrobeats     82
0      00s        ambient      2
21     10s        ambient     42
49     20s        ambient    315
22     10s         arabic     45
..     ...            ...    ...
79     20s       wellness     80
20     00s          world     17
47     10s          world     76
80     20s          world    127
118    90s          world      8

[119 rows x 3 columns]


## Visual style

In [11]:
#paper easthetic
PAPER_BG = '#f0e6d2'  # Old paper color
INK_COLOR = '#2c2c2c' # Dark grey/black for text
FONT_FAMILY = "Garamond, 'Times New Roman', serif"

In [12]:
# Custom visual theme function for Plotly
def style_fig(fig):
    fig.update_layout(
        paper_bgcolor=PAPER_BG,
        plot_bgcolor=PAPER_BG,
        font=dict(family=FONT_FAMILY, color=INK_COLOR),
        title_font=dict(size=20, family=FONT_FAMILY),
        margin=dict(t=50, l=20, r=20, b=20)
    )
    return fig

## Figures

In [14]:
selected_decade = '10s' # <--- for now, will change later
decade_data = genre_counts[genre_counts['decade'] == selected_decade]

###  FIGURE 1: rising genres (Biggest Increase) 

In [15]:

# Get top 5 genres with highest positive change
top_risers = decade_data[decade_data['change'] > 0].nlargest(5, 'change')

if not top_risers.empty:
    fig_risers = px.bar(
        top_risers, 
        x='playlist_genre', 
        y='change',
        title=f"Fastest Growing Genres in the {selected_decade}",
        text='change',
        color_discrete_sequence=['#2E8B57'] # Green for growth
    )
    # Apply your Paper Theme
    style_fig(fig_risers)
else:
    # Handle cases (like the 1960s) where there is no "previous" decade to compare to
    fig_risers = px.bar(title="No previous data to calculate growth")
    style_fig(fig_risers)

###  FIGURE 2: Declining genres (Biggest decrease) 

In [28]:

# Get bottom 5 genres (lowest negative change)
top_fallers = decade_data[decade_data['change'] < 0].nsmallest(5, 'change')

if not top_fallers.empty:
    # We make the values positive for the chart so bars go up, but label them negative
    top_fallers['display_change'] = top_fallers['change'].abs()
    
    fig_fallers = px.bar(
        top_fallers, 
        x='playlist_genre', 
        y='display_change',
        title=f"Biggest Decline in the {selected_decade}",
        text='change',
        color_discrete_sequence=['#A52A2A'] # Red/Brown for decline
    )
    style_fig(fig_fallers)
else:
    fig_fallers = px.bar(title="No significant decline detected")
    style_fig(fig_fallers)

### FIGURE 3: Decade commentary

In [40]:
#decade descrition
DECADE_INFO = {
    '1960s': "The swinging sixties. Rock evolved from rock 'n' roll, and the British Invasion changed music forever.",
    '1970s': "The era of disco, punk, and the rise of heavy metal. Music became a powerful force for political expression.",
    '1980s': "Synthesizers and drum machines took over. Pop icons like MJ and Madonna defined the MTV generation.",
    '1990s': "Grunge, hip-hop, and the boy band craze. The digital era began to reshape how we consumed music.",
    '2000s': "The new millennium brought the internet, piracy, and a blending of genres like never before.",
    '2010s': "Streaming services democratized music access. EDM and Latin pop went global.",
    '2020s': "The viral era. TikTok trends dictate hits, and short-form content rules the charts."
}


In [42]:
from dash import Dash, html
import pandas as pd

# 1. colours
# Define your colors and text again just to be safe
DECADE_COLORS = {
    '60s': '#D35400', # Burnt Orange
    '70s': '#8E44AD', # Purple
    '80s': '#2980B9', # Blue
    '90s': '#C0392B', # Red
    '00s': '#27AE60', # Green
    '10s': '#F1C40F', # Gold
    '20s': '#1ABC9C'  # Teal
}

In [44]:
def create_decade_card(decade):
    color = DECADE_COLORS.get(decade, '#333')
    text = DECADE_INFO.get(decade, "Description unavailable.")
    
    return html.Div(style={
        'fontFamily': "Garamond, 'Times New Roman', serif",
        'backgroundColor': '#fff',
        'border': '1px solid #ccc',
        'boxShadow': '5px 5px 10px rgba(0,0,0,0.1)', # Subtle shadow for depth
        'maxWidth': '500px', # Restrict width like a real card
        'margin': '20px auto' # Center it
    }, children=[
        
        # A. The Colored Header (The "Tab")
        html.Div(style={
            'backgroundColor': color,
            'height': '15px', # The colored strip at the top
            'width': '100%'
        }),
        
        # B. The Content Window
        html.Div(style={'padding': '25px'}, children=[
            # Title
            html.H2(f"The {decade}", style={
                'marginTop': '0', 
                'borderBottom': f'2px solid {color}', # Underline matches header
                'paddingBottom': '10px',
                'color': '#2c2c2c'
            }),
            
            # The Text Description
            html.P(text, style={'fontSize': '1.2em', 'lineHeight': '1.5', 'color': '#333'}),
            
            # The Hallmark Image (Placeholder for now)
            html.Div(style={
                'marginTop': '20px',
                'height': '200px',
                'backgroundColor': '#eee',
                'display': 'flex',
                'alignItems': 'center',
                'justifyContent': 'center',
                'border': '1px dashed #999'
            }, children=[
                html.Span("( Image Here)", style={'fontStyle': 'italic', 'color': '#666'})
                # Once you have real images: html.Img(src=f'assets/{decade}.jpg', style={'width': '100%'})
            ])
        ])
    ])

In [46]:
# --- 3. PREVIEW IT (The "Show" trick) ---
# We pick '10s' just to demonstrate
app = Dash(__name__)
app.layout = create_decade_card('10s')

if __name__ == '__main__':
    # This will show the card inside your notebook
    app.run(debug=True, jupyter_mode='inline', height=600)

## Dasboard

In [21]:

from dash import html

# Creates the row of buttons
decade_buttons = html.Div(
    style={'display': 'flex', 'gap': '15px', 'marginBottom': '30px', 'flexWrap': 'wrap'},
    children=[
        html.Button(
            decade_text, 
            id=f'btn-{decade_text}', # giving each button a unique ID like 'btn-90s'
            n_clicks=0,
            style={
                'padding': '15px 25px', 
                'fontSize': '1.2em', 
                'border': '2px solid #2c2c2c', # Ink color border
                'backgroundColor': '#fff', 
                'cursor': 'pointer',
                'fontFamily': FONT_FAMILY
            }
        ) for decade_text in decades_list
    ]
)